In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

from sklearn.impute import SimpleImputer

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import roc_auc_score

from sklearn.feature_selection import chi2

from sklearn.feature_selection import SelectKBest

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical

from sklearn.ensemble import RandomForestClassifier


In [2]:
df_application = pd.read_csv('application_train.csv')

In [3]:
df_application

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,F,N,N,0,112500.0,700830.0,22738.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
2,318952,0,Cash loans,M,Y,N,0,180000.0,544491.0,16047.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,3.0
3,361264,0,Cash loans,F,N,Y,0,270000.0,814041.0,28971.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
4,260639,0,Cash loans,F,N,Y,0,144000.0,675000.0,21906.0,...,0,0,0,0,0.0,0.0,0.0,10.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246003,242114,0,Cash loans,F,N,Y,1,270000.0,1172470.5,34411.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,8.0
246004,452374,0,Cash loans,F,N,Y,0,180000.0,654498.0,27859.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
246005,276545,1,Revolving loans,M,N,N,1,112500.0,270000.0,13500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
246006,236776,1,Cash loans,M,Y,N,3,202500.0,204858.0,17653.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_application.TARGET.value_counts()

0    226038
1     19970
Name: TARGET, dtype: int64

In [5]:
df_application.isna().sum()

SK_ID_CURR                        0
TARGET                            0
NAME_CONTRACT_TYPE                0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
                              ...  
AMT_REQ_CREDIT_BUREAU_DAY     33172
AMT_REQ_CREDIT_BUREAU_WEEK    33172
AMT_REQ_CREDIT_BUREAU_MON     33172
AMT_REQ_CREDIT_BUREAU_QRT     33172
AMT_REQ_CREDIT_BUREAU_YEAR    33172
Length: 122, dtype: int64

In [6]:
df_application['NAME_CONTRACT_TYPE'].describe

<bound method NDFrame.describe of 0              Cash loans
1              Cash loans
2              Cash loans
3              Cash loans
4              Cash loans
               ...       
246003         Cash loans
246004         Cash loans
246005    Revolving loans
246006         Cash loans
246007         Cash loans
Name: NAME_CONTRACT_TYPE, Length: 246008, dtype: object>

In [7]:
df_application.info

<bound method DataFrame.info of         SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0           456162       0         Cash loans           F            N   
1           134978       0         Cash loans           F            N   
2           318952       0         Cash loans           M            Y   
3           361264       0         Cash loans           F            N   
4           260639       0         Cash loans           F            N   
...            ...     ...                ...         ...          ...   
246003      242114       0         Cash loans           F            N   
246004      452374       0         Cash loans           F            N   
246005      276545       1    Revolving loans           M            N   
246006      236776       1         Cash loans           M            Y   
246007      454197       0         Cash loans           F            N   

       FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
0        

In [8]:
x = df_application.drop(['TARGET'], axis=1)
y = df_application.TARGET

In [9]:
chi2(x[['SK_ID_CURR']], y)

(array([61091.33837046]), array([0.]))

### ~Sexo~

In [10]:
pd.crosstab(df_application.TARGET, df_application.CODE_GENDER, normalize = 'columns')

CODE_GENDER,F,M,XNA
TARGET,,,
0,0.929695,0.897906,1.0
1,0.070305,0.102094,0.0


In [11]:
df_application.CODE_GENDER.value_counts(True)

F      0.657975
M      0.342013
XNA    0.000012
Name: CODE_GENDER, dtype: float64

In [12]:
df_application['CODE_GENDER'] = df_application['CODE_GENDER'].map({'F': 1, 'M': 2, 'XNA' : 3})

In [13]:
chi2(df_application[['CODE_GENDER']], y)

(array([125.72090269]), array([3.53916552e-29]))

### Possui carro

In [14]:
pd.crosstab(df_application.TARGET, df_application.FLAG_OWN_CAR, normalize = 'columns')

FLAG_OWN_CAR,N,Y
TARGET,,
0,0.914373,0.927462
1,0.085627,0.072538


In [15]:
df_application.FLAG_OWN_CAR.value_counts(True)

N    0.659958
Y    0.340042
Name: FLAG_OWN_CAR, dtype: float64

### Possui casa ou apartamento

In [16]:
pd.crosstab(df_application.TARGET, df_application.FLAG_OWN_REALTY, normalize = 'columns')

FLAG_OWN_REALTY,N,Y
TARGET,,
0,0.916142,0.920011
1,0.083858,0.079989


In [17]:
df_application.FLAG_OWN_REALTY.value_counts(True)

Y    0.693209
N    0.306791
Name: FLAG_OWN_REALTY, dtype: float64

### Quantidade de filhos

In [18]:
pd.crosstab(df_application.TARGET, df_application.CNT_CHILDREN, normalize = 'columns')

CNT_CHILDREN,0,1,2,3,4,5,6,7,8,10,11,12,14,19
TARGET,,,,,,,,,,,,,,
0,0.922492,0.909864,0.913174,0.90099,0.860606,0.909091,0.733333,1.0,1.0,1.0,0.0,1.0,1.0,1.0
1,0.077508,0.090136,0.086826,0.09901,0.139394,0.090909,0.266667,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [19]:
df_application.CNT_CHILDREN.value_counts(True)

0     0.701136
1     0.198339
2     0.086891
3     0.011906
4     0.001341
5     0.000268
6     0.000061
7     0.000016
8     0.000008
10    0.000008
12    0.000008
19    0.000008
11    0.000004
14    0.000004
Name: CNT_CHILDREN, dtype: float64

### Renda 

In [20]:
pd.crosstab(df_application.TARGET, df_application.AMT_INCOME_TOTAL, normalize = 'columns')

AMT_INCOME_TOTAL,25650.0,26100.0,26550.0,27000.0,27585.0,27900.0,27967.5,28161.0,28350.0,28404.0,...,3150000.0,3375000.0,3600000.0,3950059.5,4500000.0,6750000.0,9000000.0,13500000.0,18000090.0,117000000.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,0.959184,1.0,0.833333,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.040816,0.0,0.166667,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [21]:
df_application.AMT_INCOME_TOTAL.value_counts(True)

135000.0    0.116248
112500.0    0.101249
157500.0    0.086440
180000.0    0.080388
90000.0     0.072656
              ...   
196434.0    0.000004
198378.0    0.000004
159597.0    0.000004
560250.0    0.000004
223857.0    0.000004
Name: AMT_INCOME_TOTAL, Length: 2150, dtype: float64

### Montante de crédito

In [22]:
pd.crosstab(df_application.TARGET, df_application.AMT_CREDIT, normalize = 'columns')

AMT_CREDIT,45000.0,47970.0,48519.0,49455.0,49500.0,49752.0,50940.0,52128.0,52767.0,53910.0,...,2961000.0,2980521.0,3075579.0,3150000.0,3375000.0,3600000.0,3860019.0,4027680.0,4031032.5,4050000.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,0.947368,0.959302,1.0,0.909091,1.0,0.975,0.946667,0.977273,1.0,0.816327,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
1,0.052632,0.040698,0.0,0.090909,0.0,0.025,0.053333,0.022727,0.0,0.183673,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Preço do bem para o valor do emprestimo

In [23]:
pd.crosstab(df_application.TARGET, df_application.AMT_GOODS_PRICE, normalize = 'columns')

AMT_GOODS_PRICE,40500.0,45000.0,49500.0,50751.0,54000.0,58500.0,59782.5,60070.5,63000.0,67392.0,...,2736000.0,2745000.0,2925000.0,2961000.0,3150000.0,3375000.0,3555000.0,3600000.0,3712500.0,4050000.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.949134,0.984615,1.0,0.940426,0.92549,1.0,1.0,0.94382,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.666667,1.0,1.0
1,0.0,0.050866,0.015385,0.0,0.059574,0.07451,0.0,0.0,0.05618,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.333333,0.0,0.0


### Tipo de pessoa que acompanhou o cliente

In [24]:
pd.crosstab(df_application.TARGET, df_application.NAME_TYPE_SUITE, normalize = 'columns')

NAME_TYPE_SUITE,Children,Family,Group of people,Other_A,Other_B,"Spouse, partner",Unaccompanied
TARGET,,,,,,,
0,0.921194,0.925276,0.903226,0.911466,0.901243,0.921171,0.917671
1,0.078806,0.074724,0.096774,0.088534,0.098757,0.078829,0.082329


### Tipo de renda do cliente

In [9]:
pd.crosstab(df_application.TARGET, df_application.NAME_INCOME_TYPE, normalize = 'columns')

NAME_INCOME_TYPE,Businessman,Commercial associate,Maternity leave,Pensioner,State servant,Student,Unemployed,Working
TARGET,,,,,,,,
0,1.0,0.923955,0.666667,0.945662,0.942493,1.0,0.647059,0.903915
1,0.0,0.076045,0.333333,0.054338,0.057507,0.0,0.352941,0.096085


In [10]:
df_application.NAME_INCOME_TYPE.value_counts()

Working                 126919
Commercial associate     57321
Pensioner                44389
State servant            17337
Unemployed                  17
Student                     12
Businessman                 10
Maternity leave              3
Name: NAME_INCOME_TYPE, dtype: int64

In [13]:
# Categorizando a feture tipo de renda
def trata_acompanhante_(valor):
    if valor in ['Working']:
        return 1
    elif valor in ['Commercial associate']:
        return 2
    else:
        return 3
    
df_application['NAME_INCOME_TYPE'] = df_application['NAME_INCOME_TYPE'].apply(lambda x: trata_acompanhante_(x))

In [14]:
df_application['NAME_INCOME_TYPE'].value_counts()

1    126919
4     61768
2     57321
Name: NAME_INCOME_TYPE, dtype: int64

### Status civil do cliente

In [26]:
pd.crosstab(df_application.TARGET, df_application.NAME_FAMILY_STATUS, normalize = 'columns')

NAME_FAMILY_STATUS,Civil marriage,Married,Separated,Single / not married,Unknown,Widow
TARGET,,,,,,
0,0.899109,0.924087,0.919398,0.900673,1.0,0.941656
1,0.100891,0.075913,0.080602,0.099327,0.0,0.058344


### Tipo de moradia do cliente

In [27]:
pd.crosstab(df_application.TARGET, df_application.NAME_HOUSING_TYPE, normalize = 'columns')

NAME_HOUSING_TYPE,Co-op apartment,House / apartment,Municipal apartment,Office apartment,Rented apartment,With parents
TARGET,,,,,,
0,0.917853,0.921776,0.914257,0.93504,0.873232,0.880255
1,0.082147,0.078224,0.085743,0.06496,0.126768,0.119745


### Dias antes em que a pessoa começou a trabalhar

In [15]:
df_application['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

In [18]:
-df_application['DAYS_EMPLOYED']

0          813.0
1          223.0
2         6202.0
3            NaN
4         2763.0
           ...  
246003    4333.0
246004       NaN
246005    1982.0
246006    2840.0
246007    2084.0
Name: DAYS_EMPLOYED, Length: 246008, dtype: float64

In [16]:
pd.crosstab(df_application.TARGET, df_application.DAYS_EMPLOYED, normalize = 'columns')

DAYS_EMPLOYED,-17912.0,-17583.0,-17546.0,-17531.0,-17170.0,-17139.0,-16852.0,-16849.0,-16836.0,-16767.0,...,-11.0,-10.0,-9.0,-8.0,-6.0,-4.0,-3.0,-2.0,-1.0,0.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.333333,1.0,1.0,0.75,1.0,0.0,1.0,0.5
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.666667,0.0,0.0,0.25,0.0,1.0,0.0,0.5


### Dias antes em que a pessoa alterou seu cadastro

In [29]:
pd.crosstab(df_application.TARGET, df_application.DAYS_REGISTRATION, normalize = 'columns')

DAYS_REGISTRATION,-23738.0,-23416.0,-22858.0,-22442.0,-21887.0,-21865.0,-21249.0,-21244.0,-21146.0,-20981.0,...,-9.0,-8.0,-7.0,-6.0,-5.0,-4.0,-3.0,-2.0,-1.0,0.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.876923,0.843137,0.864865,0.8875,0.873016,0.884615,0.871429,0.828947,0.842105,0.887097
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.123077,0.156863,0.135135,0.1125,0.126984,0.115385,0.128571,0.171053,0.157895,0.112903


### Dias antes que o cliente alterou seu documento

In [30]:
pd.crosstab(df_application.TARGET, df_application.DAYS_ID_PUBLISH, normalize = 'columns')

DAYS_ID_PUBLISH,-7197,-6551,-6383,-6337,-6265,-6263,-6259,-6258,-6256,-6235,...,-9,-8,-7,-6,-5,-4,-3,-2,-1,0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.847826,0.916667,0.846154,0.955882,0.923077,0.891304,0.897436,0.894737,0.862745,0.857143
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.152174,0.083333,0.153846,0.044118,0.076923,0.108696,0.102564,0.105263,0.137255,0.142857


### Ocupação do cliente

In [31]:
pd.crosstab(df_application.TARGET, df_application.OCCUPATION_TYPE, normalize = 'columns')

OCCUPATION_TYPE,Accountants,Cleaning staff,Cooking staff,Core staff,Drivers,HR staff,High skill tech staff,IT staff,Laborers,Low-skill Laborers,Managers,Medicine staff,Private service staff,Realty agents,Sales staff,Secretaries,Security staff,Waiters/barmen staff
TARGET,,,,,,,,,,,,,,,,,,
0,0.951525,0.902913,0.895617,0.936345,0.886195,0.936652,0.939813,0.933333,0.893645,0.825677,0.937768,0.932141,0.933082,0.919094,0.903192,0.926806,0.894992,0.884686
1,0.048475,0.097087,0.104383,0.063655,0.113805,0.063348,0.060187,0.066667,0.106355,0.174323,0.062232,0.067859,0.066918,0.080906,0.096808,0.073194,0.105008,0.115314


In [32]:
df_application.OCCUPATION_TYPE.value_counts(True)

Laborers                 0.261455
Sales staff              0.152402
Core staff               0.129572
Managers                 0.100734
Drivers                  0.088718
High skill tech staff    0.053949
Accountants              0.046815
Medicine staff           0.040428
Security staff           0.031824
Cooking staff            0.027985
Cleaning staff           0.021971
Private service staff    0.012573
Low-skill Laborers       0.010061
Waiters/barmen staff     0.006423
Secretaries              0.006233
Realty agents            0.003662
HR staff                 0.002619
IT staff                 0.002577
Name: OCCUPATION_TYPE, dtype: float64

### Quantidade de membros da familia

In [33]:
pd.crosstab(df_application.TARGET, df_application.CNT_FAM_MEMBERS, normalize = 'columns')

CNT_FAM_MEMBERS,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,12.0,13.0,14.0,16.0,20.0
TARGET,,,,,,,,,,,,,,,
0,0.915904,0.923831,0.911328,0.914083,0.902697,0.853503,0.920635,0.714286,1.0,1.0,1.0,0.0,1.0,1.0,1.0
1,0.084096,0.076169,0.088672,0.085917,0.097303,0.146497,0.079365,0.285714,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Avaliação sobre a regiao onde o cliente mora

In [34]:
pd.crosstab(df_application.TARGET, df_application.REGION_RATING_CLIENT, normalize = 'columns')

REGION_RATING_CLIENT,1,2,3
TARGET,,,
0,0.951399,0.920711,0.888184
1,0.048601,0.079289,0.111816


### Avaliação sobre a regiao onde o cliente mora considerando a cidade

In [35]:
pd.crosstab(df_application.TARGET, df_application.REGION_RATING_CLIENT_W_CITY, normalize = 'columns')

REGION_RATING_CLIENT_W_CITY,1,2,3
TARGET,,,
0,0.951255,0.920473,0.884859
1,0.048745,0.079527,0.115141


### Endereço permanente do cliente não corresponde com o de contato

In [36]:
pd.crosstab(df_application.TARGET, df_application.REG_CITY_NOT_LIVE_CITY, normalize = 'columns')

REG_CITY_NOT_LIVE_CITY,0,1
TARGET,,
0,0.922437,0.876266
1,0.077563,0.123734


### Endereço comercial do cliente não corresponde com o de contato

In [37]:
pd.crosstab(df_application.TARGET, df_application.REG_CITY_NOT_WORK_CITY, normalize = 'columns')

REG_CITY_NOT_WORK_CITY,0,1
TARGET,,
0,0.926336,0.893749
1,0.073664,0.106251


### Tipo de organização onde o cliente trabalha

In [38]:
pd.crosstab(df_application.TARGET, df_application.ORGANIZATION_TYPE, normalize = 'columns')

ORGANIZATION_TYPE,Advertising,Agriculture,Bank,Business Entity Type 1,Business Entity Type 2,Business Entity Type 3,Cleaning,Construction,Culture,Electricity,...,Trade: type 4,Trade: type 5,Trade: type 6,Trade: type 7,Transport: type 1,Transport: type 2,Transport: type 3,Transport: type 4,University,XNA
TARGET,,,,,,,,,,,,,,,,,,,,,
0,0.924699,0.894388,0.94899,0.91665,0.913672,0.906285,0.899038,0.879569,0.947712,0.936141,...,0.962264,0.95122,0.952381,0.905241,0.954248,0.92149,0.840345,0.910451,0.947966,0.945539
1,0.075301,0.105612,0.05101,0.08335,0.086328,0.093715,0.100962,0.120431,0.052288,0.063859,...,0.037736,0.04878,0.047619,0.094759,0.045752,0.07851,0.159655,0.089549,0.052034,0.054461


In [39]:
pd.crosstab(df_application.TARGET, df_application.APARTMENTS_AVG, normalize = 'columns')

APARTMENTS_AVG,0.0000,0.0010,0.0014,0.0015,0.0021,0.0024,0.0026,0.0027,0.0031,0.0034,...,0.9562,0.9649,0.9660,0.9670,0.9763,0.9804,0.9814,0.9876,0.9907,1.0000
TARGET,,,,,,,,,,,,,,,,,,,,,
0,0.924749,0.940789,1.0,0.833333,0.922591,1.0,1.0,1.0,0.898693,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.920354
1,0.075251,0.059211,0.0,0.166667,0.077409,0.0,0.0,0.0,0.101307,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.079646


### Observações do entorno social do cliente 30 dias de atraso

In [40]:
pd.crosstab(df_application.TARGET, df_application.OBS_30_CNT_SOCIAL_CIRCLE, normalize = 'columns')

OBS_30_CNT_SOCIAL_CIRCLE,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,21.0,22.0,23.0,24.0,25.0,26.0,27.0,29.0,30.0,348.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,0.920263,0.91971,0.917247,0.916945,0.914001,0.915823,0.912427,0.905072,0.919115,0.918176,...,1.0,0.9375,0.846154,1.0,0.888889,1.0,1.0,1.0,0.0,1.0
1,0.079737,0.08029,0.082753,0.083055,0.085999,0.084177,0.087573,0.094928,0.080885,0.081824,...,0.0,0.0625,0.153846,0.0,0.111111,0.0,0.0,0.0,1.0,0.0


In [41]:
pd.crosstab(df_application.TARGET, df_application.DEF_30_CNT_SOCIAL_CIRCLE, normalize = 'columns')

DEF_30_CNT_SOCIAL_CIRCLE,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,34.0
TARGET,,,,,,,,,
0,0.921618,0.899428,0.88316,0.881098,0.863014,0.886364,0.888889,1.0,1.0
1,0.078382,0.100572,0.11684,0.118902,0.136986,0.113636,0.111111,0.0,0.0


### Observações do entorno social do cliente 60 dias de atraso

In [42]:
pd.crosstab(df_application.TARGET, df_application.OBS_60_CNT_SOCIAL_CIRCLE, normalize = 'columns')

OBS_60_CNT_SOCIAL_CIRCLE,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,19.0,20.0,21.0,22.0,23.0,24.0,25.0,27.0,29.0,344.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,0.920193,0.919937,0.91736,0.91677,0.913929,0.914718,0.911962,0.908128,0.918385,0.917834,...,0.825,1.0,0.96,1.0,0.8,1.0,0.888889,1.0,0.5,1.0
1,0.079807,0.080063,0.08264,0.08323,0.086071,0.085282,0.088038,0.091872,0.081615,0.082166,...,0.175,0.0,0.04,0.0,0.2,0.0,0.111111,0.0,0.5,0.0


In [43]:
pd.crosstab(df_application.TARGET, df_application.DEF_60_CNT_SOCIAL_CIRCLE, normalize = 'columns')

DEF_60_CNT_SOCIAL_CIRCLE,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,24.0
TARGET,,,,,,,,,
0,0.921116,0.895383,0.878871,0.843687,0.907563,0.933333,1.0,1.0,1.0
1,0.078884,0.104617,0.121129,0.156313,0.092437,0.066667,0.0,0.0,0.0


### Quantos dias antes o cliente alterou o telefone

In [44]:
pd.crosstab(df_application.TARGET, df_application.DAYS_LAST_PHONE_CHANGE, normalize = 'columns')

DAYS_LAST_PHONE_CHANGE,-4292.0,-4173.0,-4153.0,-4131.0,-4121.0,-4118.0,-4115.0,-4092.0,-4079.0,-4078.0,...,-9.0,-8.0,-7.0,-6.0,-5.0,-4.0,-3.0,-2.0,-1.0,0.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.92638,0.878378,0.908078,0.934579,0.942636,0.933137,0.95,0.942423,0.908931,0.903219
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.07362,0.121622,0.091922,0.065421,0.057364,0.066863,0.05,0.057577,0.091069,0.096781


### Numero de consultas ao sistema de credito uma horas antes

In [45]:
pd.crosstab(df_application.TARGET, df_application.AMT_REQ_CREDIT_BUREAU_HOUR, normalize = 'columns')

AMT_REQ_CREDIT_BUREAU_HOUR,0.0,1.0,2.0,3.0,4.0
TARGET,,,,,
0,0.922305,0.92176,0.857143,1.0,1.0
1,0.077695,0.07824,0.142857,0.0,0.0


### Numero de consultas ao sistema de credito um dia antes

In [46]:
pd.crosstab(df_application.TARGET, df_application.AMT_REQ_CREDIT_BUREAU_DAY, normalize = 'columns')

AMT_REQ_CREDIT_BUREAU_DAY,0.0,1.0,2.0,3.0,4.0,5.0,6.0,8.0,9.0
TARGET,,,,,,,,,
0,0.922409,0.898551,0.91358,0.947368,0.869565,1.0,1.0,1.0,1.0
1,0.077591,0.101449,0.08642,0.052632,0.130435,0.0,0.0,0.0,0.0


### Numero de consultas ao sistema de credito uma semana antes

In [47]:
pd.crosstab(df_application.TARGET, df_application.AMT_REQ_CREDIT_BUREAU_WEEK, normalize = 'columns')

AMT_REQ_CREDIT_BUREAU_WEEK,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
TARGET,,,,,,,,,
0,0.92227,0.923508,0.902439,0.933333,0.9,0.8,0.928571,1.0,1.0
1,0.07773,0.076492,0.097561,0.066667,0.1,0.2,0.071429,0.0,0.0


### Numero de consultas ao sistema de credito um mês antes

In [48]:
pd.crosstab(df_application.TARGET, df_application.AMT_REQ_CREDIT_BUREAU_MON, normalize = 'columns')

AMT_REQ_CREDIT_BUREAU_MON,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,14.0,15.0,16.0,17.0,18.0,19.0,22.0,23.0,24.0,27.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,0.921378,0.923506,0.929068,0.946941,0.940023,0.937107,0.940741,0.936709,0.964539,0.965116,...,1.0,0.954545,1.0,0.923077,1.0,1.0,1.0,1.0,1.0,1.0
1,0.078622,0.076494,0.070932,0.053059,0.059977,0.062893,0.059259,0.063291,0.035461,0.034884,...,0.0,0.045455,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0


### Numero de consultas ao sistema de credito três meses antes

In [49]:
pd.crosstab(df_application.TARGET, df_application.AMT_REQ_CREDIT_BUREAU_QRT, normalize = 'columns')

AMT_REQ_CREDIT_BUREAU_QRT,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,19.0
TARGET,,,,,,,,,,
0,0.921064,0.933442,0.915974,0.926087,0.883152,0.893617,0.681818,0.8,1.0,0.0
1,0.078936,0.066558,0.084026,0.073913,0.116848,0.106383,0.318182,0.2,0.0,1.0


### Numero de consultas ao sistema de credito um ano antes

In [50]:
pd.crosstab(df_application.TARGET, df_application.AMT_REQ_CREDIT_BUREAU_YEAR, normalize = 'columns')

AMT_REQ_CREDIT_BUREAU_YEAR,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,22.0,23.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,0.928439,0.926327,0.918078,0.919819,0.91775,0.915445,0.909237,0.904529,0.913742,0.882619,...,0.933333,0.666667,1.0,0.666667,1.0,1.0,1.0,1.0,0.0,1.0
1,0.071561,0.073673,0.081922,0.080181,0.08225,0.084555,0.090763,0.095471,0.086258,0.117381,...,0.066667,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,1.0,0.0


In [51]:
df_application

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,1,N,N,0,112500.0,700830.0,22738.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,134978,0,Cash loans,1,N,N,0,90000.0,375322.5,14422.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
2,318952,0,Cash loans,2,Y,N,0,180000.0,544491.0,16047.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,3.0
3,361264,0,Cash loans,1,N,Y,0,270000.0,814041.0,28971.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
4,260639,0,Cash loans,1,N,Y,0,144000.0,675000.0,21906.0,...,0,0,0,0,0.0,0.0,0.0,10.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246003,242114,0,Cash loans,1,N,Y,1,270000.0,1172470.5,34411.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,8.0
246004,452374,0,Cash loans,1,N,Y,0,180000.0,654498.0,27859.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
246005,276545,1,Revolving loans,2,N,N,1,112500.0,270000.0,13500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
246006,236776,1,Cash loans,2,Y,N,3,202500.0,204858.0,17653.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_amostra_teste = df_application[['SK_ID_CURR', 'TARGET']]

In [5]:
df_amostra_teste

,SK_ID_CURR,TARGET
0,456162,0
1,134978,0
2,318952,0
3,361264,0
4,260639,0
...,...,...
246003,242114,0
246004,452374,0
246005,276545,1
246006,236776,1


### tabela com todos os créditos anteriores dos clientes

In [6]:
df_bureau = pd.read_csv('bureau.csv')

In [7]:
df_bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [56]:
df_bureau.isna().sum()

SK_ID_CURR                      0
SK_ID_BUREAU                    0
CREDIT_ACTIVE                   0
CREDIT_CURRENCY                 0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
DAYS_CREDIT_ENDDATE        105553
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_MAX_OVERDUE    1124488
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM                 13
AMT_CREDIT_SUM_DEBT        257669
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
AMT_ANNUITY               1226791
dtype: int64

In [8]:
df_junto = pd.merge(left=df_amostra_teste, right=df_bureau, how = 'left', on = 'SK_ID_CURR')

In [9]:
df_junto

,SK_ID_CURR,TARGET,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,456162,0,5050151.0,Closed,currency 1,-352.0,0.0,378.0,-201.0,NaN,0.0,54634.500,0.00,0.000,0.0,Consumer credit,-196.0,NaN
1,456162,0,5050152.0,Closed,currency 1,-524.0,0.0,-176.0,-174.0,NaN,0.0,225000.000,0.00,NaN,0.0,Consumer credit,-174.0,NaN
2,456162,0,5050153.0,Active,currency 1,-251.0,0.0,1543.0,NaN,NaN,0.0,180000.000,182547.00,NaN,0.0,Credit card,-6.0,NaN
3,456162,0,5050154.0,Active,currency 1,-536.0,0.0,NaN,NaN,0.0,0.0,292500.000,275850.00,16645.905,0.0,Credit card,-21.0,NaN
4,456162,0,5050155.0,Closed,currency 1,-249.0,0.0,116.0,-188.0,0.0,0.0,42075.000,0.00,0.000,0.0,Consumer credit,-186.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208939,452374,0,5676418.0,Active,currency 1,-256.0,0.0,109.0,NaN,0.0,0.0,118080.000,45760.50,0.000,0.0,Consumer credit,-12.0,NaN
1208940,452374,0,5676419.0,Active,currency 1,-84.0,0.0,282.0,NaN,0.0,0.0,206480.385,176251.05,0.000,0.0,Consumer credit,-24.0,NaN
1208941,276545,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1208942,236776,1,6515867.0,Closed,currency 1,-2525.0,0.0,-2341.0,-2334.0,NaN,0.0,33057.000,0.00,NaN,0.0,Consumer credit,-513.0,NaN


In [59]:
# Status do crédito parece ser importante
pd.crosstab(df_junto.TARGET, df_junto.CREDIT_ACTIVE, normalize = 'columns')

CREDIT_ACTIVE,Active,Bad debt,Closed,Sold
TARGET,,,,
0,0.905865,0.764706,0.929792,0.896827
1,0.094135,0.235294,0.070208,0.103173


In [60]:
# moeda não parece significar tanto
df_junto.CREDIT_CURRENCY.value_counts(1)

currency 1    0.999168
currency 2    0.000733
currency 3    0.000095
currency 4    0.000004
Name: CREDIT_CURRENCY, dtype: float64

In [61]:

pd.crosstab(df_junto.TARGET, df_junto.DAYS_CREDIT, normalize = 'columns')

DAYS_CREDIT,-2922.0,-2921.0,-2920.0,-2919.0,-2918.0,-2917.0,-2916.0,-2915.0,-2914.0,-2913.0,...,-9.0,-8.0,-7.0,-6.0,-5.0,-4.0,-3.0,-2.0,-1.0,0.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,0.940217,0.923469,0.922374,0.960474,0.939914,0.941463,0.959276,0.960352,0.918552,0.933014,...,0.81407,0.820755,0.810056,0.811024,0.708861,0.792208,0.685185,0.709677,0.75,1.0
1,0.059783,0.076531,0.077626,0.039526,0.060086,0.058537,0.040724,0.039648,0.081448,0.066986,...,0.18593,0.179245,0.189944,0.188976,0.291139,0.207792,0.314815,0.290323,0.25,0.0


In [62]:
# numero de dias de atraso no crédito, não parece fazer sentido pois quanto mais se aumenta os dias de atraso pior seria 
pd.crosstab(df_junto.TARGET, df_junto.CREDIT_DAY_OVERDUE, normalize = 'columns')

CREDIT_DAY_OVERDUE,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,2684.0,2688.0,2693.0,2700.0,2754.0,2765.0,2770.0,2776.0,2781.0,2792.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,0.92108,1.0,0.5,0.888889,0.666667,0.857143,0.736842,0.79661,0.808219,0.786885,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
1,0.07892,0.0,0.5,0.111111,0.333333,0.142857,0.263158,0.20339,0.191781,0.213115,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [63]:
# duração restante do crédito
pd.crosstab(df_junto.TARGET, df_junto.DAYS_CREDIT_ENDDATE, normalize = 'columns')

DAYS_CREDIT_ENDDATE,-42056.0,-42042.0,-42013.0,-41938.0,-41905.0,-41899.0,-41880.0,-41879.0,-41878.0,-41877.0,...,31190.0,31191.0,31192.0,31193.0,31194.0,31195.0,31196.0,31197.0,31198.0,31199.0
TARGET,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.942029,0.95122,0.910112,0.933333,0.925532,0.928571,0.97561,0.86,0.96875,1.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.057971,0.04878,0.089888,0.066667,0.074468,0.071429,0.02439,0.14,0.03125,0.0


In [64]:
pd.crosstab(df_junto.TARGET, df_junto.AMT_CREDIT_MAX_OVERDUE, normalize = 'columns')

AMT_CREDIT_MAX_OVERDUE,0.000000e+00,4.500000e-02,9.000000e-02,1.350000e-01,1.800000e-01,2.250000e-01,2.700000e-01,3.150000e-01,3.600000e-01,4.050000e-01,...,6.925409e+06,7.995280e+06,8.613149e+06,9.231525e+06,1.086181e+07,1.124604e+07,1.147806e+07,1.314453e+07,1.411139e+07,1.695001e+07
TARGET,,,,,,,,,,,,,,,,,,,,,
0,0.923618,1.0,1.0,1.0,1.0,0.75,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
1,0.076382,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0


In [65]:
# quantidade de vezes que o credito foi prolongado
pd.crosstab(df_junto.TARGET, df_junto.CNT_CREDIT_PROLONG, normalize = 'columns')

CNT_CREDIT_PROLONG,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
TARGET,,,,,,,,,,
0,0.920872,0.910137,0.91155,0.975,0.947368,1.0,1.0,1.0,1.0,1.0
1,0.079128,0.089863,0.08845,0.025,0.052632,0.0,0.0,0.0,0.0,0.0


In [13]:
df_junto.CREDIT_TYPE.value_counts()

2    1132366
3      69080
1       7498
Name: CREDIT_TYPE, dtype: int64

In [14]:
# tipo do crédito
pd.crosstab(df_junto.TARGET, df_junto.CREDIT_TYPE, normalize = 'columns')

CREDIT_TYPE,1,2,3
TARGET,,,
0,0.78861,0.920965,0.92154
1,0.21139,0.079035,0.07846


In [12]:
# Categorizando a coluna CREDIT_TYPE
def trata_CREDIT_TYPE(valor):
    if valor in ['Loan for the purchase of equipment', 'Microloan', 'Loan for working capital replenishment']:
        return 1
    elif valor in ['Consumer credit', 'Credit card']:
        return 2
    else:
        return 3
    
df_junto['CREDIT_TYPE'] = df_junto['CREDIT_TYPE'].apply(lambda x: trata_CREDIT_TYPE(x))

In [67]:
# criando a coluna da duração do crédito = dias antes do pedido -  duração restante
df_junto['CREDIT_DURATION_DAYS'] = -df_junto['DAYS_CREDIT'] + df_junto['DAYS_CREDIT_ENDDATE']

In [68]:
# Categorizando a df_junto, se não deveu é 0 senão é 1
df_junto['CREDIT_DAY_OVERDUE'] = df_junto['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 0 else 0)

In [69]:
df_junto

,SK_ID_CURR,TARGET,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CREDIT_DURATION_DAYS
0,456162,0,5050151.0,Closed,currency 1,-352.0,0,378.0,-201.0,NaN,0.0,54634.500,0.00,0.000,0.0,Consumer credit,-196.0,NaN,730.0
1,456162,0,5050152.0,Closed,currency 1,-524.0,0,-176.0,-174.0,NaN,0.0,225000.000,0.00,NaN,0.0,Consumer credit,-174.0,NaN,348.0
2,456162,0,5050153.0,Active,currency 1,-251.0,0,1543.0,NaN,NaN,0.0,180000.000,182547.00,NaN,0.0,Credit card,-6.0,NaN,1794.0
3,456162,0,5050154.0,Active,currency 1,-536.0,0,NaN,NaN,0.0,0.0,292500.000,275850.00,16645.905,0.0,Credit card,-21.0,NaN,NaN
4,456162,0,5050155.0,Closed,currency 1,-249.0,0,116.0,-188.0,0.0,0.0,42075.000,0.00,0.000,0.0,Consumer credit,-186.0,NaN,365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208939,452374,0,5676418.0,Active,currency 1,-256.0,0,109.0,NaN,0.0,0.0,118080.000,45760.50,0.000,0.0,Consumer credit,-12.0,NaN,365.0
1208940,452374,0,5676419.0,Active,currency 1,-84.0,0,282.0,NaN,0.0,0.0,206480.385,176251.05,0.000,0.0,Consumer credit,-24.0,NaN,366.0
1208941,276545,1,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1208942,236776,1,6515867.0,Closed,currency 1,-2525.0,0,-2341.0,-2334.0,NaN,0.0,33057.000,0.00,NaN,0.0,Consumer credit,-513.0,NaN,184.0


In [70]:
df_junto.corr()['TARGET'].sort_values()

AMT_CREDIT_SUM           -0.011783
SK_ID_BUREAU             -0.007523
AMT_CREDIT_SUM_LIMIT     -0.005898
SK_ID_CURR               -0.002498
AMT_ANNUITY               0.000326
CNT_CREDIT_PROLONG        0.001280
AMT_CREDIT_SUM_DEBT       0.002140
AMT_CREDIT_SUM_OVERDUE    0.004567
AMT_CREDIT_MAX_OVERDUE    0.007671
CREDIT_DURATION_DAYS      0.017253
CREDIT_DAY_OVERDUE        0.018625
DAYS_CREDIT_ENDDATE       0.026427
DAYS_ENDDATE_FACT         0.040156
DAYS_CREDIT_UPDATE        0.041541
DAYS_CREDIT               0.062828
TARGET                    1.000000
Name: TARGET, dtype: float64

In [71]:
df_junto.CREDIT_DAY_OVERDUE.value_counts()

0    1205960
1       2984
Name: CREDIT_DAY_OVERDUE, dtype: int64

### Tabelas com saldos mensais de créditos anteriores

In [72]:
df_bureau_balance = pd.read_csv('bureau_balance.csv')

In [73]:
df_bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [74]:
df_bureau_balance.isna().sum()

SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [75]:
df_bureau_balance.corr()['SK_ID_BUREAU'].sort_values()

MONTHS_BALANCE    0.011873
SK_ID_BUREAU      1.000000
Name: SK_ID_BUREAU, dtype: float64

In [76]:
#df_bureau_balance.groupby(['SK_ID_BUREAU', 'MONTHS_BALANCE']).sum()

### Tabela com todos os pedidos anteriores de creditos dos clientes

In [77]:
df_previous = pd.read_csv('previous_application.csv')

In [78]:
df_previous

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.290,267295.5,311400.0,0.0,267295.5,WEDNESDAY,12,...,Furniture,30.0,low_normal,POS industry with interest,365243.0,-508.0,362.0,-358.0,-351.0,0.0
1670210,2357031,334635,Consumer loans,6622.020,87750.0,64291.5,29250.0,87750.0,TUESDAY,15,...,Furniture,12.0,middle,POS industry with interest,365243.0,-1604.0,-1274.0,-1304.0,-1297.0,0.0
1670211,2659632,249544,Consumer loans,11520.855,105237.0,102523.5,10525.5,105237.0,MONDAY,12,...,Consumer electronics,10.0,low_normal,POS household with interest,365243.0,-1457.0,-1187.0,-1187.0,-1181.0,0.0
1670212,2785582,400317,Cash loans,18821.520,180000.0,191880.0,NaN,180000.0,WEDNESDAY,9,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-1155.0,-825.0,-825.0,-817.0,1.0


In [79]:
df_previous.isna().sum()

SK_ID_PREV                           0
SK_ID_CURR                           0
NAME_CONTRACT_TYPE                   0
AMT_ANNUITY                     372235
AMT_APPLICATION                      0
AMT_CREDIT                           1
AMT_DOWN_PAYMENT                895844
AMT_GOODS_PRICE                 385515
WEEKDAY_APPR_PROCESS_START           0
HOUR_APPR_PROCESS_START              0
FLAG_LAST_APPL_PER_CONTRACT          0
NFLAG_LAST_APPL_IN_DAY               0
RATE_DOWN_PAYMENT               895844
RATE_INTEREST_PRIMARY          1664263
RATE_INTEREST_PRIVILEGED       1664263
NAME_CASH_LOAN_PURPOSE               0
NAME_CONTRACT_STATUS                 0
DAYS_DECISION                        0
NAME_PAYMENT_TYPE                    0
CODE_REJECT_REASON                   0
NAME_TYPE_SUITE                 820405
NAME_CLIENT_TYPE                     0
NAME_GOODS_CATEGORY                  0
NAME_PORTFOLIO                       0
NAME_PRODUCT_TYPE                    0
CHANNEL_TYPE             

In [80]:
df_previous.CODE_REJECT_REASON.value_counts()

XAP       1353093
HC         175231
LIMIT       55680
SCO         37467
CLIENT      26436
SCOFR       12811
XNA          5244
VERIF        3535
SYSTEM        717
Name: CODE_REJECT_REASON, dtype: int64

In [81]:
df_junto2 = pd.merge(left=df_amostra_teste, right=df_previous, how = 'left', on = 'SK_ID_CURR')

In [82]:
df_junto2

,SK_ID_CURR,TARGET,SK_ID_PREV,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,456162,0,2584213.0,Consumer loans,4960.080,38430.0,27054.0,15372.0,38430.0,SATURDAY,...,Furniture,6.0,middle,POS industry with interest,365243.0,-558.0,-408.0,-408.0,-406.0,0.0
1,134978,0,2065826.0,Consumer loans,11880.315,78732.0,66307.5,15750.0,78732.0,SUNDAY,...,Consumer electronics,6.0,low_normal,POS household with interest,365243.0,-342.0,-192.0,-222.0,-212.0,0.0
2,134978,0,1126849.0,Consumer loans,7442.280,29790.0,26122.5,4500.0,29790.0,WEDNESDAY,...,Industry,4.0,low_normal,POS industry with interest,365243.0,-2548.0,-2458.0,-2458.0,-2452.0,1.0
3,134978,0,2613419.0,Consumer loans,12240.495,112495.5,110155.5,11250.0,112495.5,THURSDAY,...,Consumer electronics,10.0,low_normal,POS household with interest,365243.0,-211.0,59.0,365243.0,365243.0,0.0
4,134978,0,1620940.0,Consumer loans,9756.135,91462.5,48762.0,45000.0,91462.5,SUNDAY,...,Consumer electronics,6.0,high,POS household with interest,365243.0,-1378.0,-1228.0,-1228.0,-1220.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1145333,454197,0,1900834.0,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1145334,454197,0,1602180.0,Cash loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1145335,454197,0,1050953.0,Consumer loans,6893.370,57600.0,62667.0,0.0,57600.0,TUESDAY,...,Furniture,10.0,low_normal,POS industry with interest,365243.0,-489.0,-219.0,-213.0,-208.0,0.0
1145336,454197,0,2828789.0,Cash loans,11765.340,81000.0,99135.0,NaN,81000.0,TUESDAY,...,XNA,12.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
df_junto2.corr()['TARGET'].sort_values()

DAYS_FIRST_DRAWING          -0.031849
HOUR_APPR_PROCESS_START     -0.028708
RATE_DOWN_PAYMENT           -0.026609
AMT_DOWN_PAYMENT            -0.018927
AMT_ANNUITY                 -0.016040
DAYS_FIRST_DUE              -0.006875
AMT_APPLICATION             -0.006694
NFLAG_LAST_APPL_IN_DAY      -0.003478
AMT_CREDIT                  -0.003311
SELLERPLACE_AREA            -0.002420
SK_ID_CURR                  -0.001595
AMT_GOODS_PRICE             -0.001058
NFLAG_INSURED_ON_APPROVAL    0.000972
SK_ID_PREV                   0.001617
RATE_INTEREST_PRIMARY        0.001980
DAYS_TERMINATION             0.016929
DAYS_LAST_DUE                0.017215
DAYS_LAST_DUE_1ST_VERSION    0.018595
CNT_PAYMENT                  0.029623
RATE_INTEREST_PRIVILEGED     0.029893
DAYS_DECISION                0.039015
TARGET                       1.000000
Name: TARGET, dtype: float64

In [84]:
df_junto2.select_dtypes(include=['object']).columns

Index(['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
       'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION'],
      dtype='object')

In [85]:
# Arrumando as variaveis de dias
df_junto2['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
df_junto2['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
df_junto2['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
df_junto2['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
df_junto2['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)

In [86]:
df_junto2.corr()['TARGET'].sort_values()

HOUR_APPR_PROCESS_START     -0.028708
RATE_DOWN_PAYMENT           -0.026609
AMT_DOWN_PAYMENT            -0.018927
AMT_ANNUITY                 -0.016040
AMT_APPLICATION             -0.006694
NFLAG_LAST_APPL_IN_DAY      -0.003478
AMT_CREDIT                  -0.003311
SELLERPLACE_AREA            -0.002420
SK_ID_CURR                  -0.001595
AMT_GOODS_PRICE             -0.001058
NFLAG_INSURED_ON_APPROVAL    0.000972
SK_ID_PREV                   0.001617
RATE_INTEREST_PRIMARY        0.001980
DAYS_TERMINATION             0.021844
DAYS_LAST_DUE                0.022275
DAYS_FIRST_DUE               0.028436
DAYS_LAST_DUE_1ST_VERSION    0.028881
CNT_PAYMENT                  0.029623
RATE_INTEREST_PRIVILEGED     0.029893
DAYS_DECISION                0.039015
DAYS_FIRST_DRAWING           0.092967
TARGET                       1.000000
Name: TARGET, dtype: float64

In [87]:
# Criando a variavel da diferença do valor que o cliente pediu para o valor que ele recebeu
df_junto2['DIFFERENCE_RECEIVED'] = df_junto2['AMT_APPLICATION'] - df_junto2['AMT_CREDIT']

In [88]:
#
pd.crosstab(df_junto2.TARGET, df_junto2.NAME_CONTRACT_TYPE, normalize = 'columns')

NAME_CONTRACT_TYPE,Cash loans,Consumer loans,Revolving loans,XNA
TARGET,,,,
0,0.908426,0.922337,0.894164,0.809717
1,0.091574,0.077663,0.105836,0.190283


In [89]:
pd.crosstab(df_junto2.TARGET, df_junto2.WEEKDAY_APPR_PROCESS_START, normalize = 'columns')

WEEKDAY_APPR_PROCESS_START,FRIDAY,MONDAY,SATURDAY,SUNDAY,THURSDAY,TUESDAY,WEDNESDAY
TARGET,,,,,,,
0,0.911601,0.911206,0.915993,0.919232,0.911174,0.911781,0.911853
1,0.088399,0.088794,0.084007,0.080768,0.088826,0.088219,0.088147


In [90]:
#
pd.crosstab(df_junto2.TARGET, df_junto2.FLAG_LAST_APPL_PER_CONTRACT, normalize = 'columns')

FLAG_LAST_APPL_PER_CONTRACT,N,Y
TARGET,,
0,0.887496,0.913065
1,0.112504,0.086935


In [91]:
df_junto2['FLAG_LAST_APPL_PER_CONTRACT'] = df_junto2['FLAG_LAST_APPL_PER_CONTRACT'].map({'Y': 1, 'N': 0})

In [92]:
# objetivo do emprestimo - problemas com XAP e XNA
pd.crosstab(df_junto2.TARGET, df_junto2.NAME_CASH_LOAN_PURPOSE, normalize = 'columns')

NAME_CASH_LOAN_PURPOSE,Building a house or an annex,Business development,Buying a garage,Buying a holiday home / land,Buying a home,Buying a new car,Buying a used car,Car repairs,Education,Everyday expenses,...,Money for a third person,Other,Payments on other loans,Purchase of electronic equipment,Refusal to name the goal,Repairs,Urgent needs,Wedding / gift / holiday,XAP,XNA
TARGET,,,,,,,,,,,,,,,,,,,,,
0,0.861629,0.880435,0.944444,0.888,0.866551,0.90634,0.871147,0.817029,0.891144,0.899752,...,0.823529,0.873509,0.846154,0.877384,0.666667,0.870168,0.849171,0.875394,0.916517,0.912523
1,0.138371,0.119565,0.055556,0.112,0.133449,0.09366,0.128853,0.182971,0.108856,0.100248,...,0.176471,0.126491,0.153846,0.122616,0.333333,0.129832,0.150829,0.124606,0.083483,0.087477


In [93]:
def trata_NAME_CASH(valor):
    if valor in ['Refusal to name the goal']:
        return 1
    elif valor in ['Car repairs', 'Money for a third person', 'Gasification / water supply', 'Hobby']:
        return 2
    elif valor in ['Payments on other loans', 'Urgent needs']:
        return 3
    elif valor in ['Building a house or an annex', 'Buying a home', 'Repairs', 'Buying a used car', 'Other',
                   'Purchase of electronic equipment', 'Wedding / gift / holiday', 'Buying a holiday home / land', 'Medicine']:
        return 4
    elif valor in ['Business development', 'Education', 'Everyday expenses', 'Furniture', 'Journey']:
        return 5
    elif valor in ['XAP', 'XNA', 'Buying a new car', 'Buying a garage']:
        return 6
    else:
        return valor
    
df_junto2['NAME_CASH_LOAN_PURPOSE'] = df_junto2['NAME_CASH_LOAN_PURPOSE'].apply(lambda x: trata_NAME_CASH(x))

In [94]:
df_junto2.corr()['TARGET'].sort_values()

NAME_CASH_LOAN_PURPOSE        -0.033987
HOUR_APPR_PROCESS_START       -0.028708
RATE_DOWN_PAYMENT             -0.026609
AMT_DOWN_PAYMENT              -0.018927
AMT_ANNUITY                   -0.016040
DIFFERENCE_RECEIVED           -0.012504
AMT_APPLICATION               -0.006694
FLAG_LAST_APPL_PER_CONTRACT   -0.006531
NFLAG_LAST_APPL_IN_DAY        -0.003478
AMT_CREDIT                    -0.003311
SELLERPLACE_AREA              -0.002420
SK_ID_CURR                    -0.001595
AMT_GOODS_PRICE               -0.001058
NFLAG_INSURED_ON_APPROVAL      0.000972
SK_ID_PREV                     0.001617
RATE_INTEREST_PRIMARY          0.001980
DAYS_TERMINATION               0.021844
DAYS_LAST_DUE                  0.022275
DAYS_FIRST_DUE                 0.028436
DAYS_LAST_DUE_1ST_VERSION      0.028881
CNT_PAYMENT                    0.029623
RATE_INTEREST_PRIVILEGED       0.029893
DAYS_DECISION                  0.039015
DAYS_FIRST_DRAWING             0.092967
TARGET                         1.000000


In [95]:
#
pd.crosstab(df_junto2.TARGET, df_junto2.NAME_CONTRACT_STATUS, normalize = 'columns')

NAME_CONTRACT_STATUS,Approved,Canceled,Refused,Unused offer
TARGET,,,,
0,0.92346,0.907423,0.880309,0.917154
1,0.07654,0.092577,0.119691,0.082846


In [96]:
pd.crosstab(df_junto2.TARGET, df_junto2.NAME_PAYMENT_TYPE, normalize = 'columns')

NAME_PAYMENT_TYPE,Cash through the bank,Cashless from the account of the employer,Non-cash from your account,XNA
TARGET,,,,
0,0.918619,0.915119,0.912206,0.903341
1,0.081381,0.084881,0.087794,0.096659


In [97]:
pd.crosstab(df_junto2.TARGET, df_junto2.CODE_REJECT_REASON, normalize = 'columns')

CODE_REJECT_REASON,CLIENT,HC,LIMIT,SCO,SCOFR,SYSTEM,VERIF,XAP,XNA
TARGET,,,,,,,,,
0,0.917154,0.882891,0.87502,0.899298,0.793774,0.929348,0.911741,0.91983,0.895951
1,0.082846,0.117109,0.12498,0.100702,0.206226,0.070652,0.088259,0.08017,0.104049


In [98]:
pd.crosstab(df_junto2.TARGET, df_junto2.NAME_CLIENT_TYPE, normalize = 'columns')

NAME_CLIENT_TYPE,New,Refreshed,Repeater,XNA
TARGET,,,,
0,0.909776,0.92611,0.912297,0.889248
1,0.090224,0.07389,0.087703,0.110752


In [99]:
#
pd.crosstab(df_junto2.TARGET, df_junto2.NAME_PORTFOLIO, normalize = 'columns')

NAME_PORTFOLIO,Cards,Cars,Cash,POS,XNA
TARGET,,,,,
0,0.897245,0.948387,0.910837,0.923094,0.90217
1,0.102755,0.051613,0.089163,0.076906,0.09783


In [100]:
pd.crosstab(df_junto2.TARGET, df_junto2.NAME_YIELD_GROUP, normalize = 'columns')

NAME_YIELD_GROUP,XNA,high,low_action,low_normal,middle
TARGET,,,,,
0,0.900771,0.903728,0.93555,0.928262,0.919211
1,0.099229,0.096272,0.06445,0.071738,0.080789


### Tabela com balanço mensal de crédito em dinheiro

In [101]:
df_CASH = pd.read_csv('POS_CASH_balance.csv')

In [102]:
df_CASH

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,Active,843,0
10001354,1717234,141565,-19,12.0,0.0,Active,602,0
10001355,1283126,315695,-21,10.0,0.0,Active,609,0
10001356,1082516,450255,-22,12.0,0.0,Active,614,0


In [103]:
df_CASH.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
MONTHS_BALANCE               0
CNT_INSTALMENT           26071
CNT_INSTALMENT_FUTURE    26087
NAME_CONTRACT_STATUS         0
SK_DPD                       0
SK_DPD_DEF                   0
dtype: int64

In [104]:
df_junto3 = pd.merge(left=df_amostra_teste, right=df_CASH, how = 'left', on = 'SK_ID_CURR')

In [105]:
df_junto3

,SK_ID_CURR,TARGET,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,456162,0,2584213.0,-14.0,6.0,1.0,Active,0.0,0.0
1,456162,0,2584213.0,-16.0,6.0,3.0,Active,0.0,0.0
2,456162,0,2584213.0,-19.0,6.0,6.0,Active,0.0,0.0
3,456162,0,2584213.0,-18.0,6.0,5.0,Active,0.0,0.0
4,456162,0,2584213.0,-17.0,6.0,4.0,Active,0.0,0.0
...,...,...,...,...,...,...,...,...,...
6861990,454197,0,1050953.0,-12.0,11.0,5.0,Active,0.0,0.0
6861991,454197,0,1050953.0,-7.0,11.0,0.0,Completed,0.0,0.0
6861992,454197,0,1050953.0,-10.0,11.0,3.0,Active,0.0,0.0
6861993,454197,0,1050953.0,-8.0,11.0,1.0,Active,0.0,0.0


In [106]:
df_junto3['SK_DPD'].value_counts()

0.0       6639797
1.0         15152
2.0         12193
3.0         10176
4.0          8636
           ...   
3086.0          1
2999.0          1
3089.0          1
3610.0          1
2652.0          1
Name: SK_DPD, Length: 3320, dtype: int64

In [107]:
# Categorizando, se deve = 1 senão = 0
df_junto3['SK_DPD'] = df_junto3['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)

In [108]:
df_junto3['SK_DPD'].value_counts()

0    6654133
1     207862
Name: SK_DPD, dtype: int64

In [109]:
df_junto3

,SK_ID_CURR,TARGET,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,456162,0,2584213.0,-14.0,6.0,1.0,Active,0,0.0
1,456162,0,2584213.0,-16.0,6.0,3.0,Active,0,0.0
2,456162,0,2584213.0,-19.0,6.0,6.0,Active,0,0.0
3,456162,0,2584213.0,-18.0,6.0,5.0,Active,0,0.0
4,456162,0,2584213.0,-17.0,6.0,4.0,Active,0,0.0
...,...,...,...,...,...,...,...,...,...
6861990,454197,0,1050953.0,-12.0,11.0,5.0,Active,0,0.0
6861991,454197,0,1050953.0,-7.0,11.0,0.0,Completed,0,0.0
6861992,454197,0,1050953.0,-10.0,11.0,3.0,Active,0,0.0
6861993,454197,0,1050953.0,-8.0,11.0,1.0,Active,0,0.0


In [110]:
df_junto3.corr()['TARGET'].sort_values()

SK_ID_CURR              -0.003157
SK_ID_PREV              -0.000115
SK_DPD_DEF               0.004524
SK_DPD                   0.015390
CNT_INSTALMENT           0.017868
MONTHS_BALANCE           0.019486
CNT_INSTALMENT_FUTURE    0.021592
TARGET                   1.000000
Name: TARGET, dtype: float64

In [111]:
pd.crosstab(df_junto3.TARGET, df_junto3.NAME_CONTRACT_STATUS, normalize= 'columns')

NAME_CONTRACT_STATUS,Active,Amortized debt,Approved,Canceled,Completed,Demand,Returned to the store,Signed,XNA
TARGET,,,,,,,,,
0,0.92605,0.771704,0.927225,0.833333,0.924842,0.881186,0.895737,0.921905,1.0
1,0.07395,0.228296,0.072775,0.166667,0.075158,0.118814,0.104263,0.078095,0.0


### Tabela de historico de amortização de credito residencial

In [112]:
df_instalments = pd.read_csv('installments_payments.csv')

In [113]:
df_instalments

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,NaN,67.500,NaN
13605397,1310347,414406,0.0,47,-1539.0,NaN,67.500,NaN
13605398,1308766,402199,0.0,43,-7.0,NaN,43737.435,NaN
13605399,1062206,409297,0.0,43,-1986.0,NaN,67.500,NaN


In [114]:
df_instalments.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64

In [115]:
# Criando a variavel de qual o valor que o cliente realmente pagou
df_instalments['REALLY_PAID_AMOUNT'] = df_instalments['AMT_INSTALMENT'] - df_instalments['AMT_PAYMENT']

In [116]:
# Criando a variavel da deferença do dia do pagamento
df_instalments['REALLY_PAID_DAY'] = df_instalments['DAYS_INSTALMENT'] - df_instalments['DAYS_ENTRY_PAYMENT']

In [117]:
df_instalments

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,REALLY_PAID_AMOUNT,REALLY_PAID_DAY
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,0.000,7.0
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,0.000,0.0
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,0.000,0.0
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,0.000,8.0
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,4.455,-17.0
...,...,...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,NaN,67.500,NaN,NaN,NaN
13605397,1310347,414406,0.0,47,-1539.0,NaN,67.500,NaN,NaN,NaN
13605398,1308766,402199,0.0,43,-7.0,NaN,43737.435,NaN,NaN,NaN
13605399,1062206,409297,0.0,43,-1986.0,NaN,67.500,NaN,NaN,NaN


### Tabela mensal de informações do cartão de crédito

In [118]:
df_credit = pd.read_csv('credit_card_balance.csv')

In [132]:
df_credit

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,NaN,0.0,NaN,NaN,0.000,...,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0
3840308,1714892,347207,-9,0.000,45000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,23.0,Active,0,0
3840309,1302323,215757,-9,275784.975,585000,270000.0,270000.0,0.0,0.0,2250.000,...,273093.975,273093.975,2.0,2,0.0,0.0,18.0,Active,0,0
3840310,1624872,430337,-10,0.000,450000,NaN,0.0,NaN,NaN,0.000,...,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0


In [120]:
df_credit.isna().sum()

SK_ID_PREV                         0
SK_ID_CURR                         0
MONTHS_BALANCE                     0
AMT_BALANCE                        0
AMT_CREDIT_LIMIT_ACTUAL            0
AMT_DRAWINGS_ATM_CURRENT      749816
AMT_DRAWINGS_CURRENT               0
AMT_DRAWINGS_OTHER_CURRENT    749816
AMT_DRAWINGS_POS_CURRENT      749816
AMT_INST_MIN_REGULARITY       305236
AMT_PAYMENT_CURRENT           767988
AMT_PAYMENT_TOTAL_CURRENT          0
AMT_RECEIVABLE_PRINCIPAL           0
AMT_RECIVABLE                      0
AMT_TOTAL_RECEIVABLE               0
CNT_DRAWINGS_ATM_CURRENT      749816
CNT_DRAWINGS_CURRENT               0
CNT_DRAWINGS_OTHER_CURRENT    749816
CNT_DRAWINGS_POS_CURRENT      749816
CNT_INSTALMENT_MATURE_CUM     305236
NAME_CONTRACT_STATUS               0
SK_DPD                             0
SK_DPD_DEF                         0
dtype: int64

In [121]:
df_credit.SK_DPD.value_counts()

0       3686957
1         90369
8          2772
32         2340
7          1797
         ...   
1501          1
2647          1
1520          1
2649          1
2831          1
Name: SK_DPD, Length: 917, dtype: int64

In [122]:
# Categorizando, se deve = 1 senão = 0
df_credit['SK_DPD'] = df_credit['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)

In [123]:
df_credit.SK_DPD.value_counts()

0    3686957
1     153355
Name: SK_DPD, dtype: int64

In [124]:
# Criando a variavel de credito usado
#df_credit['LIMIT_USED'] = df_credit['AMT_CREDIT_LIMIT_ACTUAL'] - df_credit['AMT_BALANCE']

In [125]:
df_credit.drop(['SK_DPD', 'SK_DPD_DEF', 'NAME_CONTRACT_STATUS',
                'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 
                'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM'], axis=1)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.00,1800.000,0.000,0.000,0.000
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.00,2250.000,60175.080,64875.555,64875.555
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.00,2250.000,26926.425,31460.085,31460.085
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.00,11925.000,224949.285,233048.970,233048.970
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.00,27000.000,443044.395,453919.455,453919.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000
3840308,1714892,347207,-9,0.000,45000,0.0,0.0,0.0,0.0,0.000,1879.11,0.000,0.000,0.000,0.000
3840309,1302323,215757,-9,275784.975,585000,270000.0,270000.0,0.0,0.0,2250.000,375750.00,356994.675,269356.140,273093.975,273093.975
3840310,1624872,430337,-10,0.000,450000,NaN,0.0,NaN,NaN,0.000,NaN,0.000,0.000,0.000,0.000


In [126]:
#df_credit['BALANCE_USED'] = df_credit['AMT_BALANCE'] - df_credit['']

In [127]:
i = df_credit[df_credit['SK_ID_CURR']==371185]

In [128]:
i.drop(['SK_DPD', 'SK_DPD_DEF', 'NAME_CONTRACT_STATUS',
                'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 
                'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM'], axis=1)[:10]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE
2,1740877,371185,-7,31815.225,450000,0.0,0.00,0.0,0.00,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085
11335,1740877,371185,-6,26281.800,450000,0.0,0.00,0.0,0.00,2250.000,4500.0,4500.0,24384.645,26376.300,26376.300
93345,1740877,371185,-22,134691.795,450000,0.0,0.00,0.0,0.00,7013.565,9000.0,9000.0,131271.345,135847.170,135847.170
226862,1740877,371185,-26,100204.785,450000,0.0,0.00,0.0,0.00,5157.990,5400.0,5400.0,97759.845,101099.790,101099.790
554266,1740877,371185,-29,102745.215,450000,0.0,0.00,0.0,0.00,5277.870,5400.0,5400.0,100157.130,103648.140,103648.140
617894,1740877,371185,-1,141940.260,162000,0.0,113969.43,0.0,113969.43,2250.000,2250.0,2250.0,141011.550,141940.260,141940.260
690953,1740877,371185,-36,86971.005,112500,76500.0,87210.00,0.0,10710.00,3921.750,4050.0,4050.0,85094.550,87560.055,87560.055
1126096,1740877,371185,-35,84621.060,112500,0.0,0.00,0.0,0.00,4402.035,5850.0,5850.0,82190.475,85406.805,85406.805
1275289,1740877,371185,-21,132384.285,450000,0.0,0.00,0.0,0.00,6816.780,7200.0,7200.0,129135.150,133525.575,133525.575
1364594,1740877,371185,-8,35120.610,450000,36000.0,36782.10,0.0,782.10,11994.705,247500.0,247500.0,29176.425,33775.110,33775.110
